In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("7a").getOrCreate()
file_location = "online_retail.csv"
file_type = "csv"
infer_schema = "False"
first_row_is_header = "True"
df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.option("header", first_row_is_header) \
.load(file_location)
df.printSchema()
print(df.dtypes)
print(df.count())
print('The total number of records: '+str(df.count()))

22/10/16 20:40:23 WARN Utils: Your hostname, m0 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/16 20:40:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/16 20:40:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
22/10/16 20:40:31 WARN Utils: Service 'SparkUI' could not bind on

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)

[('InvoiceNo', 'string'), ('StockCode', 'string'), ('Description', 'string'), ('Quantity', 'string'), ('InvoiceDate', 'string'), ('UnitPrice', 'string'), ('CustomerID', 'string'), ('Country', 'string')]


541909


The total number of records: 541909


In [6]:
dedupe_data = df.drop_duplicates(["InvoiceNo", "InvoiceDate", "StockCode"])
dedupe_data.show()
print('The total number of records (dropped duplicates): '+str(dedupe_data.count()))

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|    21730|GLASS STAR FROSTE...|       6|01/12/10 08:26|     4.25|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01/12/10 08:26|     7.65|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/10 08:26|     2.75|     17850|United Kingdom|
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/10 08:26|     2.

The total number of records (dropped duplicates): 531232


In [ ]:
from pyspark.sql.functions import to_timestamp, col
from pyspark.sql.types import FloatType
interim_data = (select_data
                  .withColumn("invoice_time", to_timestamp("InvoiceDate|", 'dd/M/yy HH:mm'))
                  .withColumn("cust_age", col("age").cast(FloatType()))
                  .withColumn("working_class", col("work_class").cast(FloatType()))
                  .withColumn("fin_wt", col("final_weight").cast(FloatType()))
                  .drop("final_weight")
             )
interim_data.display()

In [ ]:
# Databricks notebook source
# MAGIC %md Data Loading

# COMMAND ----------

# MAGIC %fs ls dbfs:/FileStore/shared_uploads/delta/retail_silver.delta

# COMMAND ----------

raw_data = spark.read.format("delta").load("dbfs:/FileStore/shared_uploads/delta/retail_silver.delta")
raw_data.where("gender is not null").display()

# COMMAND ----------

# MAGIC %md Data Selection

# COMMAND ----------

select_data = (raw_data
                .where("age is not null")
                .select("invoice_num", "stock_code","quantity", "invoice_date", 
                              "unit_price","country_code","age", "gender", "occupation", "work_class", "final_weight", "description"))
select_data.display()

# COMMAND ----------

# MAGIC %md Calculating basic Statistics

# COMMAND ----------

select_data.describe().display()

# COMMAND ----------

# MAGIC %md Data Cleansing

# COMMAND ----------

dedupe_data = select_data.drop_duplicates(["invoice_num", "invoice_date", "stock_code"])
dedupe_data.display()

# COMMAND ----------

# MAGIC %md Change data types

# COMMAND ----------

from pyspark.sql.functions import to_timestamp, col
from pyspark.sql.types import FloatType
interim_data = (select_data
                  .withColumn("invoice_time", to_timestamp("invoice_date", 'dd/M/yy HH:mm'))
                  .withColumn("cust_age", col("age").cast(FloatType()))
                  .withColumn("working_class", col("work_class").cast(FloatType()))
                  .withColumn("fin_wt", col("final_weight").cast(FloatType()))
                  .drop("final_weight")
             )
interim_data.display()

# COMMAND ----------

# MAGIC %md Replace NULL and NA values

# COMMAND ----------

clean_data = interim_data.na.fill(0)
clean_data.display()

# COMMAND ----------

# MAGIC %md Data Manipulation

# COMMAND ----------

final_data = (clean_data.where("age is not null")
                       .withColumnRenamed("working_class", "work_type")
                       .drop("age")
                       .drop("work_class")
                       .drop("fn_wt")
             )
final_data.display()

# COMMAND ----------

#%fs rm -r /FileStore/shared_uploads/delta/retail_ml.delta

# COMMAND ----------

(final_data.where("description is NOT NULL")
       .write.format("delta").mode("overwrite")
         .save("dbfs:/FileStore/shared_uploads/delta/retail_ml.delta"))

# COMMAND ----------

final_data.count()

# COMMAND ----------

# MAGIC %md Convert Spark DataFrame to a Pandas Dataframe

# COMMAND ----------

pd_data = final_data.toPandas()
pd_data